In [17]:
def generate_prompt(question):
    prompt_template = """
    Answer the following question with a detailed explanation:
    Question: {question}
    """
    return prompt_template.format(question=question)

In [63]:
import dotenv
from anthropic import AsyncAnthropic, RateLimitError, APIError
import pandas as pd
import os

df = pd.read_csv("./pending.csv")[["Sub Task Id", "User Query"]]
questions = list(df.itertuples(index=False, name=None))

dotenv.load_dotenv()
client = AsyncAnthropic(api_key=os.environ["CLAUDE_KEY"])
await client.beta.messages.batches.create(
    requests=[
        {
            "custom_id": str(q_id),
            "params": {
                "model": "claude-3-7-sonnet-latest",
                "max_tokens": 16384,
                "thinking": {"type": "enabled", "budget_tokens": 15000},
                "messages": [{"role": "user", "content": generate_prompt(question)}],
            },
        } for q_id, question in questions
    ]
)

BetaMessageBatch(id='msgbatch_01QKiHRJLaSHGQ4Mp51z6FmK', archived_at=None, cancel_initiated_at=None, created_at=datetime.datetime(2025, 4, 2, 13, 7, 23, 756800, tzinfo=datetime.timezone.utc), ended_at=None, expires_at=datetime.datetime(2025, 4, 3, 13, 7, 23, 756800, tzinfo=datetime.timezone.utc), processing_status='in_progress', request_counts=BetaMessageBatchRequestCounts(canceled=0, errored=0, expired=0, processing=20, succeeded=0), results_url=None, type='message_batch')

In [114]:
result_stream = await client.beta.messages.batches.results("msgbatch_01QKiHRJLaSHGQ4Mp51z6FmK")
final_responses = {}
async for entry in result_stream:
    if entry.result.type == "succeeded":
        q_id = int(entry.custom_id)
        for block in entry.result.message.content:
            if block.type == "text":
                response = block.text
        final_responses[q_id] = response

In [116]:
df["Sonnet Response"] = df["Sub Task Id"].map(final_responses)

In [118]:
df.to_csv("sonnet.csv")